# 3. Creating Dataset for Imputation

In this part, we firstly check for the linear combination in our dataset that stems from the accounting relationships among the firm characteristics, such as *financial_revenu* - *financial_expense* = *financial PL*, i.e., we look for the A+B=C relationships among varibales based on our accounting knowledge. For the identified relationships we remove the C variables. Further, we check whether the discovered relationships hold throughout the dataset. We find out that it is **not** the case, there are two issues: firstly, sometimes A and B are non missing while C is missing, this issue is not relevant for us as we remove the C variables (we did not discover cases, when A and C are non-missing or B and C are non-missing, which would allow to compute B or A from the other values); secondly, in some cases the A+B=C relationship does not hold (some errors might be due to the fat finger phenomenon, other appear to be at ranodm). Since, we are not able to determine in which variable (A or B or C) the error occur, we assume the errors are part of the C variable that is removed. 

Once the C variables are removed, we proceed by removing columns with a lot of missing values. Firstly, all *second* and *third lags* are removed as they contain a large number of defaults. Secondly, we remove columns that have more than aprox. 33\% of missing values. This leavs us dataset with dimension of 1 075 926 x 63, about 20\% of the dataset are missing values. **We use this dataset for the MICE imputation.**

In [2]:
import pandas as pd
import numpy  as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import sklearn.pipeline
import sklearn
import xgboost as xgb
import warnings
import math

# Loading Dataset

In [4]:
data = pd.read_pickle('Erasmus_data_stresstesting_2024.pickle')

In [5]:
data

,index,bvd_id_number,country_code,industry_code,size_class,status_year,status_date_latest,status_latest,default_indicator,fixed_assets_0,...,cash_flow_2,cash_flow_3,added_value_0,added_value_1,added_value_2,added_value_3,ebitda_0,ebitda_1,ebitda_2,ebitda_3
0,396160676,DE7350012856,DE,G,SME,2020,16991231,non_default,0,72912.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,409479565,RU78171352,RU,G,SME,2011,16991231,non_default,0,3193.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,409936846,RU79023978,RU,G,SME,2013,16991231,non_default,1,18743.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,409872267,RU79859926,RU,F,SME,2013,16991231,non_default,0,1328937.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,440112490,RU89954117,RU,G,SME,2014,16991231,non_default,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526973,212124085,LV000384403,LV,G,SME,2010,16991231,non_default,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,192111.0,42807.0,117551.0,NaN
1526974,316233678,DE2050497257,DE,L,SME,2020,16991231,non_default,0,24010.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526975,395041837,NL27104129,NL,G,SME,2005,19820408,non_default,0,2552433.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526976,454534921,RU67448158,RU,F,SME,2016,20161014,non_default,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Remove C variables of identified A+B=C reationships

In [6]:
# Based on accounting relation between variables in the dataset, we decided to remove these variables
columns_to_remove = [
    'fixed_assets_0', 'current_assets_0', 'total_assets_0', 'shareholders_funds_0', 
    'noncurrent_liabilities_0', 'current_liabilities_0', 'total_shareh_funds_liab_0', 
    'working_capital_0', 'net_current_assets_0', 'operating_revenue_0', 'financial_pl_0', 
    'pl_before_tax_0', 'pl_after_tax_0', 'pl_for_period_net_income_0', 'ebitda_0'
]

# Drop the specified columns
df = data.drop(columns=columns_to_remove) 

# remove also these variable with lag 1
columns_to_remove = [col.replace('_0', '_1') for col in columns_to_remove]

df = df.drop(columns=columns_to_remove)



In [7]:
# Next we remove all variables with lag 2 and 3 as they have too many missing values
filtered_columns = [col for col in df.columns if all(str(num) not in col for num in range(2, 4))]
df = df[filtered_columns]
df

,index,bvd_id_number,country_code,industry_code,size_class,status_year,status_date_latest,status_latest,default_indicator,intangible_fixed_assets_0,...,depreciation_amortization_0,depreciation_amortization_1,interest_paid_0,interest_paid_1,research_development_expenses_0,research_development_expenses_1,cash_flow_0,cash_flow_1,added_value_0,added_value_1
0,396160676,DE7350012856,DE,G,SME,2020,16991231,non_default,0,2905.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,409479565,RU78171352,RU,G,SME,2011,16991231,non_default,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,409936846,RU79023978,RU,G,SME,2013,16991231,non_default,1,0.0,...,NaN,NaN,362387.0,326723.0,NaN,NaN,NaN,NaN,NaN,NaN
3,409872267,RU79859926,RU,F,SME,2013,16991231,non_default,0,0.0,...,NaN,NaN,354.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,440112490,RU89954117,RU,G,SME,2014,16991231,non_default,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526973,212124085,LV000384403,LV,G,SME,2010,16991231,non_default,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526974,316233678,DE2050497257,DE,L,SME,2020,16991231,non_default,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526975,395041837,NL27104129,NL,G,SME,2005,19820408,non_default,0,0.0,...,295020.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526976,454534921,RU67448158,RU,F,SME,2016,20161014,non_default,0,0.0,...,NaN,NaN,17314.0,28545.0,NaN,NaN,NaN,NaN,NaN,NaN


# Check if the accounting relations always hold, i.e. A + B = C

In [8]:
def a_b_c(data, column1, column2, column3, type = '-'):

    for i in range(data.shape[0]):
        a = data.iloc[i, :][column1]
        if type == '+':
            b = -data.iloc[i, :][column2]
        else:
            b = data.iloc[i, :][column2]
        c = data.iloc[i, :][column3]
        
        if (math.isnan(a) == math.isnan(b) == math.isnan(c)) == True:
            x = a - b - c < 5
            diff = a - b - c

            if x == False:
                print(f'Wrong SUM with diff: {diff} at {i}: {a}, {b}, {c}')
        else:
            print(f'Missing Val: {i}: {math.isnan(a)}, {math.isnan(b)}, {math.isnan(c)}')

In [9]:
def a_b_c_d(data, column1, column2, column3, column4):

    for i in range(data.shape[0]):
        a = data.iloc[i, :][column1]
        b = data.iloc[i, :][column2]
        c = data.iloc[i, :][column3]
        d = data.iloc[i, :][column4]
        
        if (math.isnan(a) == math.isnan(b) == math.isnan(c) == math.isnan(d)) == True:
            x = a - b - c - d < 5
            diff = a - b - c - d 

            if x == False:
                print(f'Wrong SUM with diff: {diff}: {i}: {a}, {b}, {c}, {d}')
        else:
            print(f'Missing Val at {i}: {math.isnan(a)}, {math.isnan(b)}, {math.isnan(c)}, {math.isnan(d)}')

In [10]:
#check for selected columns

a_b_c(data[data['country_code'] == 'IT'], 'financial_revenue_0', 'financial_expenses_0', 'financial_pl_0')
#a_b_c(data[data['country_code'] == 'IT'], 'pl_before_tax_0', 'taxation_0', 'pl_after_tax_0')
#a_b_c(data[data['country_code'] == 'IT'], 'operating_pl_ebit_0', 'financial_pl_0', 'pl_before_tax_0', type = '+')
#a_b_c(data, 'shareholders_funds_0', 'capital_0', 'other_shareholder_funds_0')
#a_b_c_d(data, 'current_liabilities_0', 'loans_0', 'creditors_0', 'other_current_liabilities_0')
#a_b_c(data, 'noncurrent_liabilities_0', 'long_term_debt_0', 'other_noncurrent_liabilities_0')
#a_b_c_d(data, 'current_assets_0', 'stock_0', 'debtors_0', 'other_current_assets_0')

Wrong SUM with diff: 164.0 at 4133: 2164.0, 243442.0, -241442.0
Missing Val: 4184: True, False, False
Wrong SUM with diff: 127109.0 at 4911: 141746.0, 205570.0, -190933.0
Wrong SUM with diff: 4538933.0 at 6130: 4539465.0, 9766.0, -9234.0
Missing Val: 8677: True, False, False
Wrong SUM with diff: 5695914.0 at 10462: 5705656.0, 106394.0, -96652.0
Wrong SUM with diff: 9738.0 at 12867: 10500.0, 95512.0, -94750.0
Missing Val: 14754: True, True, False
Missing Val: 14781: True, True, False
Wrong SUM with diff: 19958.0 at 19338: 30000.0, 14921.0, -4879.0
Missing Val: 20715: True, True, False
Missing Val: 21125: True, True, False
Missing Val: 21142: True, True, False
Missing Val: 21840: True, True, False
Missing Val: 21870: True, True, False
Missing Val: 21871: True, True, False
Wrong SUM with diff: 10.0 at 26455: 0.0, 0.0, -10.0
Wrong SUM with diff: 16.0 at 26771: 9963994.0, 146963744.0, -136999760.0
Wrong SUM with diff: 90271.0 at 29482: 0.0, 0.0, -90271.0
Wrong SUM with diff: 950000.0 at 316

In [11]:
a_b_c_d(data[data['country_code'] == 'IT'], 'current_assets_0', 'stock_0', 'debtors_0', 'other_current_assets_0')

Wrong SUM with diff: 1000.0: 132: 13633000.0, 6795000.0, 1690000.0, 5147000.0
Wrong SUM with diff: 7.0: 1386: 175457056.0, 161764992.0, 1117304.0, 12574753.0
Wrong SUM with diff: 10.0: 1552: 186447072.0, 173945968.0, 3741105.0, 8759989.0
Wrong SUM with diff: 6.0: 3966: 159550672.0, 143981696.0, 7859807.0, 7709163.0
Wrong SUM with diff: 8.0: 5039: 138000896.0, 95148184.0, 37532360.0, 5320344.0
Wrong SUM with diff: 5.0: 7539: 82256600.0, 79472320.0, 83739.0, 2700536.0
Wrong SUM with diff: 6.0: 14526: 87259200.0, 76474744.0, 5749441.0, 5035009.0
Missing Val at 14754: True, True, False, False
Missing Val at 14781: True, True, False, False
Wrong SUM with diff: 1000.0: 16451: 2516000.0, 45000.0, 1024000.0, 1446000.0
Missing Val at 20715: True, True, False, False
Missing Val at 21125: True, True, False, False
Missing Val at 21142: True, True, False, False
Missing Val at 21840: True, True, False, False
Missing Val at 21870: True, True, False, False
Missing Val at 21871: True, True, False, Fals

In [12]:
# Now we subset only on the countries of interest based defined by Zanders
countries_of_interest = ['FI', 'NL', 'AT', 'BE', 'SE', 'DE', 'NO', 'DK', 'IS', 'IT', 'ES', 'PT', 'LV', 'RO', 'HR', 'LT', 'BG', 'SK', 'CZ', 'SI', 'HU', 'PL']

# Subset the DataFrame based on countries of interest
subset_df = df[df['country_code'].isin(countries_of_interest)]


In [13]:
# Unconditional probability of default of the dataset with only EU countries
sum(subset_df['default_indicator'])/subset_df.shape[0]

0.006265300773473269

In [14]:
# number of observations 
subset_df.shape[0]

1075926

In [15]:
subset_df

,index,bvd_id_number,country_code,industry_code,size_class,status_year,status_date_latest,status_latest,default_indicator,intangible_fixed_assets_0,...,depreciation_amortization_0,depreciation_amortization_1,interest_paid_0,interest_paid_1,research_development_expenses_0,research_development_expenses_1,cash_flow_0,cash_flow_1,added_value_0,added_value_1
0,396160676,DE7350012856,DE,G,SME,2020,16991231,non_default,0,2905.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,448629119,ESF34251967,ES,A,SME,2021,20140104,non_default,0,0.0,...,NaN,11282.0,1078.0,456.0,NaN,NaN,154.0,11381.0,34323.0,40281.0
9,7000375,IT00174710145,IT,C,SME,2006,16991231,non_default,0,819427.0,...,136504.0,109947.0,10463.0,8737.0,NaN,NaN,197115.0,220223.0,814785.0,834961.0
10,7000376,IT00174710145,IT,C,SME,2005,16991231,non_default,0,651498.0,...,109947.0,160501.0,8737.0,7114.0,NaN,NaN,220223.0,263278.0,834961.0,832154.0
11,7000905,IT02969620133,IT,G,SME,2019,16991231,non_default,0,3546.0,...,71708.0,57089.0,14449.0,7399.0,NaN,NaN,634701.0,338217.0,1313014.0,772696.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526971,430303751,IT00811420629,IT,G,SME,2006,16991231,non_default,0,24082.0,...,33524.0,30781.0,1480.0,2079.0,NaN,NaN,215824.0,348545.0,594147.0,786761.0
1526972,67988054,LV010340417,LV,F,SME,2018,16991231,non_default,0,32131.0,...,NaN,NaN,39008.0,32506.0,NaN,NaN,NaN,NaN,NaN,NaN
1526973,212124085,LV000384403,LV,G,SME,2010,16991231,non_default,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526974,316233678,DE2050497257,DE,L,SME,2020,16991231,non_default,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Lastly we drop the variables that have more than 350_000 nans in no lage, i.e. name_of_variable_0
# Next we remove lags 2 and 3 as they have too many missing values

filtered_columns = [col for col in subset_df.columns if all(str(num) not in col for num in range(1, 4))]
aux_df = subset_df[filtered_columns]
l = aux_df.columns[aux_df.isna().sum() < 350000]

# also keep their 1st lags
lags_to_keep = pd.Index([col.replace('_0', '_1') for col in l[9:]])
keep = l.append(lags_to_keep)

subset_df = subset_df[keep]


In [17]:
keep

Index(['index', 'bvd_id_number', 'country_code', 'industry_code', 'size_class',
       'status_year', 'status_date_latest', 'status_latest',
       'default_indicator', 'intangible_fixed_assets_0',
       'tangible_fixed_assets_0', 'other_fixed_assets_0', 'stock_0',
       'debtors_0', 'other_current_assets_0', 'cash_cash_equivalent_0',
       'capital_0', 'other_shareholders_funds_0', 'long_term_debt_0',
       'other_noncurrent_liabilities_0', 'provisions_0', 'loans_0',
       'creditors_0', 'other_current_liabilities_0', 'number_of_employees_0',
       'sales_0', 'operating_pl_ebit_0', 'financial_revenue_0',
       'financial_expenses_0', 'taxation_0', 'material_costs_0',
       'costs_of_employees_0', 'depreciation_amortization_0',
       'interest_paid_0', 'cash_flow_0', 'added_value_0',
       'intangible_fixed_assets_1', 'tangible_fixed_assets_1',
       'other_fixed_assets_1', 'stock_1', 'debtors_1',
       'other_current_assets_1', 'cash_cash_equivalent_1', 'capital_1',
      

# This dataset is used for imputation of missing values


In [ ]:
subset_df

In [24]:
# proportion of missing values
subset_df.isna().sum().sum()/ (1075926  * 63)

0.21271703969491734